# Wind data analysis

For this project, we will be working with the Risø dataset. The data can be found 
[here](https://data.dtu.dk/articles/dataset/Wind_resource_data_from_the_tall_Ris_met_mast/14153204).

Before we can start, we must first load all the libraries we will be using in
this notebook so that others can easily see a list of all packages needed to 
run the code in this notebook. 

After importing all the necessary libraries, we must then prepare the data for
analysis, this means loading the data, cleaning it and checking for any mistakes
or missing data.

Finally, we will proceed with the rest of the assignment.

In [ ]:
from datetime import datetime, timedelta

import pandas as pd
import numpy as np
import netCDF4 as nc

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

## Definitions
Before we actually process the data, we must define what files we're working with.
These variables can be changed to work with different datasets as long as the 
data is in a similar format. For example, they can be changed to work with the
Børglum dataset if needed. These variables are all that would need to be changed.

To bear in mind: The notebook assumes that the provided signals are in the same
order, so, if replacing these variables make sure the signals are at the same
index (e.g. the second signal in the list must be the wind speed, etc...).

In [ ]:
meso_file = 'data/risoe/meso_Risoe.csv'
mast_file = 'data/risoe/risoe_m_all.nc'

meso_signals = ['TIMESTAMP', 'WSP080', 'WDIR080']
mast_signals = ['time', 'ws77', 'wd77']

base_date = datetime(1995, 11, 20, 16, 25, 0)

## Data Preprocessing
We will start by loading the necessary data and preparing to work with it.

For this, we need both the mast data and the meso data. We will only be considering
the wind speed and its direction for this assignment. There are other indicators
that we consider have a negligible impact in the decision of the wind turbine
placement.

Since we have multiple signals to choose from, we've selected the following to
work with in this assignment:
- Mast:
    - Wind Speead at an altitude of 77m
    - Wind Direction at an altitude of 77m
- Meso:
    - Wind Speed at an altitude of 80m
    - Wind Direction at an altitude of 80m

These signals are as close to each other as possible so we believe the difference
in altitude will not have a significant impact in the data, it's only a 3m difference.


In [ ]:
# Defining a function that splices the data into the necessary columns
def preprocessing_data(data, time_signal, speed_signal, direction_signal):
    df = pd.DataFrame()
    df['TIMESTAMP'] = pd.to_datetime(data[time_signal])
    df['SPEED'] = data[speed_signal]
    df['DIRECTION'] = data[direction_signal]
    direction_radians = np.radians(df['DIRECTION'])
    df['u'] = np.sin(direction_radians)
    df['v'] = np.cos(direction_radians)
    df = df.resample('1h', on='TIMESTAMP').mean()
    df['DIRECTION'] = (np.degrees(np.arctan2(df['u'], df['v'])) + 360) % 360
    df = df.resample('1h').asfreq(1).reset_index()
    return df

In [ ]:
meso_set = pd.read_csv(meso_file)[meso_signals]

# meso_timeframe = pd.to_datetime(meso_set['TIMESTAMP'])
# display(meso_set[['TIMESTAMP', meso_signals[1]]])
meso_data = preprocessing_data(meso_set, meso_signals[0], meso_signals[1], meso_signals[2])

print(meso_data['DIRECTION'].min(), meso_data['DIRECTION'].max())
print(meso_data['SPEED'].min(), meso_data['SPEED'].max())
meso_data

In [ ]:
mast_set = nc.Dataset(mast_file, 'r')

time = []
for minutes in np.array(mast_set.variables[mast_signals[0]]):
    time_delta = timedelta(minutes=int(minutes))
    timestamp = base_date + time_delta
    time.append(timestamp)    
mast_df = pd.DataFrame()
mast_df['TIMESTAMP'] = time
mast_df['SPEED'] = np.array(mast_set.variables[mast_signals[1]])
mast_df['DIRECTION'] = np.array(mast_set.variables[mast_signals[2]])

mast_data = preprocessing_data(mast_df, 'TIMESTAMP', 'SPEED', 'DIRECTION')

print(mast_data['TIMESTAMP'].min(), mast_data['TIMESTAMP'].max())
mast_data

In [ ]:
mast_data = mast_data.dropna().reset_index()
meso_data = meso_data.dropna().reset_index()

In [ ]:
# Because there is some periodicity to the wind, we can add some features to help the model make better predictions
meso_data['HOUR_SIN'] = np.sin(2 * np.pi * meso_data['TIMESTAMP'].dt.hour / 24)
meso_data['HOUR_COS'] = np.cos(2 * np.pi * meso_data['TIMESTAMP'].dt.hour / 24)
meso_data['DAY_SIN'] = np.sin(2 * np.pi * meso_data['TIMESTAMP'].dt.dayofyear / 365)
meso_data['DAY_COS'] = np.cos(2 * np.pi * meso_data['TIMESTAMP'].dt.dayofyear / 365)

# We convert the mast data to UTC time
if 'time_utc' in mast_data.columns:
    mast_data.drop(columns=['TIMESTAMP_UTC'], inplace=True)
mast_data.insert(1, 'TIMESTAMP_UTC', mast_data['TIMESTAMP'] - pd.Timedelta(hours=1))

In [ ]:
# Check for unique years in the data
mast_data_years = mast_data['TIMESTAMP'].dt.year.unique()
meso_data_years = meso_data['TIMESTAMP'].dt.year.unique()
print(mast_data_years)
print(meso_data_years)
# Calculate the intersection of the two year ranges
overlap = np.intersect1d(mast_data_years, meso_data_years)
print(overlap)

mast_data = mast_data[mast_data['TIMESTAMP'].dt.year.isin(overlap)].reset_index()
meso_data = meso_data[meso_data['TIMESTAMP'].dt.year.isin(overlap)].reset_index()
display(mast_data, meso_data)

In [ ]:
# We can now merge the data into our final dataset
dataset = pd.merge(meso_data, mast_data, left_on='TIMESTAMP', right_on='TIMESTAMP_UTC', how='inner', suffixes=('_meso', '_mast'))
print('corr utc:', dataset['SPEED_mast'].corr(dataset['SPEED_meso']))
dataset = dataset.drop(columns=['TIMESTAMP_meso', 'level_0_meso', 'level_0_mast', 'index_mast', 'index_meso', 'TIMESTAMP_mast'])
dataset

In [ ]:
# We can now finally split the data and train a model
X = dataset[['SPEED_meso', 'DIRECTION_meso', 'HOUR_SIN', 'HOUR_COS', 'DAY_SIN', 'DAY_COS']].copy()
y = dataset[['SPEED_mast', 'u_mast', 'v_mast']].copy()

idx = int(len(dataset) * 0.8)
X_train, y_train, X_test, y_test = X[:idx], y[:idx], X[idx:], y[idx:]

X_scaler = StandardScaler()
X_train = X_scaler.fit_transform(X_train)
X_test = X_scaler.transform(X_test)
y_scaler = StandardScaler()
y_train = y_scaler.fit_transform(y_train)
y_test = y_scaler.transform(y_test)

model = Ridge()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

mse, mae, r2 = mean_squared_error(y_test[:, 0], y_pred[:, 0]), mean_absolute_error(y_test[:, 0], y_pred[:, 0]), r2_score(y_test[:, 0], y_pred[:, 0])
print(f"Wind speed\nMSE: {mse:.4f}, MAE: {mae:.4f}, R2: {r2:.4f}")

mse, mae, r2 = mean_squared_error(y_test[:, 1:], y_pred[:, 1:]), mean_absolute_error(y_test[:, 1:], y_pred[:, 1:]), r2_score(y_test[:, 1:], y_pred[:, 1:])
print(f"\nWind direction\nMSE: {mse:.4f}, MAE: {mae:.4f}, R2: {r2:.4f}")